In [1]:
%load_ext line_profiler

In [2]:
from collections import Counter
from functools import partial

import numpy as np
rng = np.random.default_rng()
np.set_printoptions(suppress=True, linewidth=150, edgeitems=5)
from tqdm.notebook import tqdm, trange
from tqdm.contrib.concurrent import process_map

from scipy.stats import rankdata

In [22]:
# ns = [0,10,10,11,12,13,14,15,16,17]
ns = [0,5,5,6,7,8,9,10,11,12,13,14,15,16,17,]


students = np.logspace(-1, 1, 11)
display(students)


# ms = 5
# arrs = np.triu(np.tile(np.arange(32, 37),(5,1)).T) - 20
# display(arrs)
# display(arrs.sum())

# ms = 4
# arrs = np.triu(np.tile(np.arange(31, 35),(4,1)).T) - 20
# display(arrs)
# display(arrs.sum())


ms = 2
arrs = np.array([[10,-20],[7,3]])
display(arrs)
display(arrs.sum())

array([ 0.1       ,  0.15848932,  0.25118864,  0.39810717,  0.63095734,  1.        ,  1.58489319,  2.51188643,  3.98107171,  6.30957344, 10.        ])

array([[ 10, -20],
       [  7,   3]])

0

In [12]:
def softmax(x, b, axis=-1):
    x = x - np.max(x)
    return np.exp(b*x) / np.exp(b*x).sum(axis=axis, keepdims=True)

def make_model(num_nodes=128, num_dense=4):
    
    import tensorflow as tf
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)
    
    inputs = tf.keras.layers.Input(shape=(ms*ms + ms + 1))
    x = tf.keras.layers.Flatten()(inputs)
    
    for _ in range(num_dense):
        x = tf.keras.layers.Dense(num_nodes, activation='relu')(x)
    
    # output1 = tf.keras.layers.Dense(5, name='Y0')(x)
    # output2 = tf.keras.layers.Dense(5, name='Y1')(x)
    # model = tf.keras.models.Model(inputs=inputs, outputs=[output1, output2])
    
    output1 = tf.keras.layers.Dense(ms, name='Y0')(x)
    output1 = tf.keras.layers.Softmax()(output1)
    model = tf.keras.models.Model(inputs=inputs, outputs=output1)

    
    opt = tf.keras.optimizers.Adam()
    # opt = tf.keras.optimizers.Adam(learning_rate=1)
    # opt = tf.keras.optimizers.SGD()
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])
    return model

def softmax(x, b, axis=-1):
    x = x - np.max(x)
    return np.exp(b*x) / np.exp(b*x).sum(axis=axis, keepdims=True)

def make_x():
    x = arrs
    
    rand = np.random.random(arrs.shape)
    rand = (rand - 0.5)
    x = x + rand
    
    x = rng.permuted(x, axis=-1)
    return x

def eval_model(model, n = 1, disable=True):
    rewards = []
    branch_rewards = []
    for _ in trange(n, disable=disable):
        x = make_x()
        x_tree = np.concatenate([np.eye(ms+1).astype(int), np.tile(x.flatten(), (ms+1,1))], axis=1)
        y_pred = model(x_tree)
        branch_reward = (x_tree[0,ms+1:].reshape(ms,ms) * y_pred[1:].numpy()).sum(axis=1)
        rewards.append(y_pred[0].numpy())
        # reward = (y_pred[0] * branch_reward).numpy().sum()
        # rewards.append(reward)
        branch_rewards.append(branch_reward)
    return np.array(rewards), np.array(branch_rewards)

def make_data(b, n=1, disable=True):
    X, Y = [], []
    
    
    for _ in trange(n, disable=disable):
        x = make_x()
        r1 = softmax(rankdata(x,axis=1), b)
        r1sum = (r1*x).sum(axis=1)
        r0 = softmax(rankdata(r1sum), b)

        pos = [0] * (ms+1)
        pos[0] = 1
        x0 = pos + list(x.flatten())
        y0 = rng.choice(np.arange(ms), p=r0)
        
        pos = [0] * (ms+1)
        pos[y0+1] = 1
        x1 = pos + list(x.flatten())
        y1 = rng.choice(np.arange(ms), p=r1[y0])


        X.append(x0)
        Y.append(y0)
        X.append(x1)
        Y.append(y1)
    
    X = np.array(X)
    Y = np.array(Y)
    
    return X,Y

In [13]:
def train_model(student):
    verbose = False
    if student == students[len(students)//2]:
        print(f'student == {student:.3f}')
        verbose = True
    model = make_model()
    Xtrain, Ytrain = make_data(student, 2**18, disable=not verbose)
    model.fit(Xtrain, Ytrain, verbose=verbose)
    reward = eval_model(model, 10000, disable=not verbose)
    model.save(f'models/starting_{student:.3f}.keras')
    return reward

In [14]:
%%time
rewards = process_map(train_model, students, disable=True, max_workers=16)

student == 1.000


  0%|          | 0/262144 [00:00<?, ?it/s]

16384/16384 [==============================] - 81s 5ms/step - loss: 0.5867 - accuracy: 0.7280


  0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: user 1.93 s, sys: 1.79 s, total: 3.72 s
Wall time: 3min 56s


In [15]:
for r, s in zip(rewards, students):
    
    res = r[0]*r[1]
    res = res.sum(axis=1).mean()
    
    print(f"{s:.3f} {res}")

0.100 0.2517535120562212
0.158 0.7234076181503103
0.251 1.7799424849739445
0.398 2.4258585762728195
0.631 3.3130862349333126
1.000 4.746010133126799
1.585 5.993108255331436
2.512 8.157832896322281
3.981 9.433404246392719
6.310 9.959263177781263
10.000 9.999053134943907


In [16]:
%%time
for b in students:
    r0s = []
    r1s = []
    for _ in trange(10000, disable=True):
        x = make_x()
        r1 = (softmax(x, b) * x).sum(axis=1)
        r2 = (softmax(r1, b) * r1).sum()
        
        r1 = softmax(rankdata(x,axis=1), b)
        r1sum = (r1*x).sum(axis=1)
        r0 = softmax(rankdata(r1sum), b)
        r0sum = (r0*r1sum).sum()

        
        r0s.append(r0sum)
        r1s.append(r1sum)
        
    r0s = np.array(r0s)
    r1s = np.array(r1s)
    print(f"{b:.3f}, {r0s.mean():.3f}, {r1s.mean(axis=0)}")


0.100, 0.659, [-4.25173511  5.1024228 ]
0.158, 1.027, [-3.81619281  5.15966067]
0.251, 1.582, [-3.13012816  5.24725861]
0.398, 2.401, [-2.05422377  5.39302992]
0.631, 3.515, [-0.41704509  5.60700897]
1.000, 4.852, [1.93444603 5.92552585]
1.585, 6.076, [4.89573619 6.31740951]
2.512, 7.671, [7.74985935 6.69977709]
3.981, 9.407, [9.45302497 6.93083964]
6.310, 9.940, [9.94556603 6.98951317]
10.000, 9.995, [9.99560377 7.00635102]
CPU times: user 32.6 s, sys: 43.4 ms, total: 32.7 s
Wall time: 32.7 s


In [17]:
def eval_training(student, teacher, verbose=False):
    import tensorflow as tf
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)

    
    if student == -1:
        model2 = make_model()
    else:
        model2 = tf.keras.models.load_model(f"models/starting_{student:.3f}.keras")
    opt = tf.keras.optimizers.SGD()
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model2.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])
    # print(f"{b:.2f}", end=':    ')
    nsum = 0
    
    X, Y = currs[f"{teacher:.3f}"]
    perm = np.random.permutation(len(X))
    X, Y = X[perm], Y[perm]
    
    rewards = []
    for i in ns:
        if i != 0:
            n = 2**i
            # X, Y = make_data(teacher, n, disable=True)
                        
            model2.fit(X[nsum:nsum+n], Y[nsum:nsum+n], verbose=False)
            nsum += n
            
        r0, r1 = eval_model(model2, 50)
        reward = (r0*r1).sum(axis=1).mean()
        rewards.append(reward)
        # print(f"{nsum}    ", end='\r')
    return rewards

In [27]:
def exp(student, verbose=True, n=20):
    # teachers = np.arange(student, 0.40001, 0.02)
    
    teachers = students
    
    t_ = list(teachers) * n
    s_ = [student for _ in t_]
    
    res = process_map(eval_training, s_, t_, max_workers=32, chunksize=1)
    
    return np.array(res)

#     all_res = []
#     for teacher in tqdm(teachers):
        
#         if teacher == teachers[0] and verbose:
#             print(f'Original Student: {student:.3f}')
#             print()
#             print('         Curriculum Size')
#             print('Teacher', end='')
#             nsum = 0
#             for i in ns:
#                 if i: nsum += 2**i
#                 print(f"{nsum:>8}", end='')
#             print()
        
#         # print(f"teacher = {teacher:.2f} ", end='\r')
#         res = process_map(eval_training, [student]*n, [teacher]*n, max_workers=10, disable=True)
#         res = np.array(res).mean(axis=0)
#         all_res.append(res)
        
#         if verbose:
#             print(f"  {teacher:.3f}", end='  ')
#             for reward in res:
#                 print(f"{reward: 6.2f}", end='  ')
#             print()

#     all_res = np.array(all_res)

    # return all_res

In [26]:
ntotal = (2**np.array(ns[1:])).sum()

currs = process_map(make_data, students, [ntotal]*len(students), max_workers=6, chunksize=1)
currs = dict(zip([f"{s:.3f}" for s in students], currs))

  0%|          | 0/11 [00:00<?, ?it/s]

In [28]:
%%time
# compiled
foo_new = exp(-1, n=100)
foo_new.reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/1100 [00:00<?, ?it/s]

CPU times: user 1.67 s, sys: 532 ms, total: 2.21 s
Wall time: 1h 9min 59s


array([[-0.11,  0.49,  0.1 ,  0.03,  0.39,  0.18,  0.09,  0.28,  0.29,  0.34,  0.32,  0.32,  0.3 ,  0.35,  0.38],
       [ 0.09,  0.36,  0.27,  0.42,  0.41,  0.52,  0.57,  0.53,  0.48,  0.49,  0.51,  0.48,  0.53,  0.56,  0.64],
       [ 0.22,  0.85,  0.43,  0.66,  1.01,  0.38,  0.46,  0.36,  0.62,  0.71,  0.72,  0.8 ,  0.84,  0.96,  1.12],
       [-0.  ,  0.87,  0.71,  0.58,  1.15,  0.75,  0.71,  0.88,  0.95,  1.02,  1.05,  1.1 ,  1.4 ,  1.67,  1.88],
       [-0.05,  1.56,  1.05,  1.03,  1.51,  1.14,  1.25,  1.28,  1.24,  1.42,  1.69,  1.9 ,  2.33,  2.7 ,  3.13],
       [ 0.25,  1.84,  1.43,  1.54,  1.76,  1.53,  1.71,  1.72,  1.75,  2.01,  2.38,  2.79,  3.58,  4.25,  4.63],
       [-0.28,  1.84,  1.69,  1.99,  2.02,  2.14,  2.36,  2.36,  2.44,  2.79,  3.29,  4.03,  4.91,  5.63,  5.94],
       [ 0.27,  1.45,  2.92,  2.63,  2.66,  3.15,  3.3 ,  3.21,  3.32,  3.92,  4.75,  6.44,  7.18,  7.27,  7.62],
       [-0.19,  2.29,  3.54,  3.29,  4.  ,  4.13,  4.26,  4.61,  4.39,  4.85,  6.11,  8.

In [29]:
%%time
# compiled
foo0100 = exp(0.1, n=100)
foo0100.reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/1100 [00:00<?, ?it/s]

CPU times: user 1.67 s, sys: 610 ms, total: 2.28 s
Wall time: 1h 10min 3s


array([[ 0.24,  0.24,  0.26,  0.25,  0.25,  0.26,  0.26,  0.25,  0.26,  0.25,  0.26,  0.24,  0.23,  0.22,  0.23],
       [ 0.24,  0.24,  0.26,  0.25,  0.26,  0.27,  0.27,  0.27,  0.3 ,  0.3 ,  0.33,  0.33,  0.33,  0.34,  0.35],
       [ 0.24,  0.24,  0.26,  0.26,  0.26,  0.28,  0.29,  0.3 ,  0.36,  0.39,  0.45,  0.47,  0.48,  0.52,  0.52],
       [ 0.24,  0.24,  0.26,  0.26,  0.27,  0.29,  0.32,  0.36,  0.45,  0.53,  0.64,  0.69,  0.72,  0.77,  0.78],
       [ 0.24,  0.24,  0.26,  0.26,  0.28,  0.32,  0.37,  0.45,  0.59,  0.73,  0.92,  1.01,  1.07,  1.13,  1.19],
       [ 0.24,  0.24,  0.27,  0.27,  0.3 ,  0.35,  0.44,  0.56,  0.8 ,  1.04,  1.32,  1.45,  1.48,  1.63,  2.17],
       [ 0.24,  0.24,  0.27,  0.28,  0.32,  0.39,  0.53,  0.72,  1.06,  1.4 ,  1.77,  1.89,  2.  ,  2.89,  5.58],
       [ 0.24,  0.23,  0.24,  0.21,  0.16,  0.09, -0.07, -0.33, -0.65, -0.68,  0.98,  2.85,  4.94,  7.22,  7.28],
       [ 0.24,  0.23,  0.23,  0.21,  0.16,  0.07, -0.1 , -0.38, -0.72, -0.6 ,  2.12,  3.

In [30]:
%%time
# compiled
foo0158 = exp(0.158, n=100)
foo0158.reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/1100 [00:00<?, ?it/s]

CPU times: user 1.77 s, sys: 622 ms, total: 2.39 s
Wall time: 1h 9min 53s


array([[ 0.72,  0.72,  0.73,  0.72,  0.72,  0.72,  0.71,  0.69,  0.69,  0.66,  0.65,  0.61,  0.59,  0.58,  0.58],
       [ 0.72,  0.72,  0.73,  0.72,  0.72,  0.73,  0.72,  0.72,  0.74,  0.76,  0.81,  0.84,  0.87,  0.89,  0.89],
       [ 0.72,  0.72,  0.73,  0.73,  0.73,  0.74,  0.75,  0.77,  0.83,  0.92,  1.08,  1.24,  1.31,  1.35,  1.36],
       [ 0.72,  0.72,  0.73,  0.73,  0.74,  0.76,  0.79,  0.84,  0.96,  1.17,  1.53,  1.85,  2.  ,  2.  ,  2.02],
       [ 0.72,  0.72,  0.73,  0.74,  0.75,  0.79,  0.85,  0.95,  1.18,  1.58,  2.25,  2.8 ,  2.9 ,  2.92,  2.91],
       [ 0.72,  0.73,  0.74,  0.75,  0.77,  0.83,  0.94,  1.11,  1.49,  2.19,  3.36,  3.94,  3.96,  3.97,  4.03],
       [ 0.72,  0.73,  0.74,  0.76,  0.8 ,  0.88,  1.04,  1.31,  1.86,  2.94,  4.58,  4.98,  5.12,  5.67,  5.97],
       [ 0.72,  0.71,  0.71,  0.7 ,  0.66,  0.62,  0.54,  0.54,  1.29,  3.09,  3.25,  3.48,  6.26,  7.46,  7.45],
       [ 0.72,  0.71,  0.71,  0.7 ,  0.66,  0.62,  0.55,  0.63,  2.43,  4.07,  4.22,  4.

In [31]:
%%time
# compiled
foo0251 = exp(0.251, n=100)
foo0251.reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/1100 [00:00<?, ?it/s]

CPU times: user 1.8 s, sys: 591 ms, total: 2.39 s
Wall time: 1h 9min 56s


array([[1.76, 1.77, 1.78, 1.77, 1.75, 1.74, 1.69, 1.6 , 1.5 , 1.33, 1.14, 0.93, 0.79, 0.68, 0.65],
       [1.77, 1.77, 1.78, 1.77, 1.76, 1.75, 1.71, 1.65, 1.57, 1.45, 1.32, 1.18, 1.08, 1.04, 1.02],
       [1.77, 1.77, 1.78, 1.77, 1.77, 1.77, 1.75, 1.72, 1.69, 1.65, 1.63, 1.59, 1.57, 1.57, 1.59],
       [1.77, 1.77, 1.78, 1.78, 1.78, 1.8 , 1.81, 1.82, 1.88, 1.97, 2.12, 2.26, 2.34, 2.37, 2.39],
       [1.77, 1.77, 1.79, 1.79, 1.81, 1.85, 1.9 , 1.99, 2.19, 2.48, 2.95, 3.32, 3.45, 3.47, 3.49],
       [1.77, 1.78, 1.8 , 1.81, 1.84, 1.91, 2.03, 2.24, 2.66, 3.33, 4.28, 4.69, 4.77, 4.8 , 4.8 ],
       [1.77, 1.78, 1.8 , 1.83, 1.88, 1.99, 2.2 , 2.57, 3.28, 4.49, 5.64, 5.93, 6.01, 6.03, 6.05],
       [1.77, 1.76, 1.76, 1.73, 1.68, 1.58, 1.4 , 1.13, 0.84, 1.21, 2.15, 3.51, 7.62, 7.59, 7.62],
       [1.77, 1.76, 1.76, 1.73, 1.67, 1.57, 1.38, 1.1 , 0.84, 2.52, 4.  , 5.44, 9.19, 9.29, 9.33],
       [1.77, 1.76, 1.75, 1.73, 1.67, 1.56, 1.36, 1.09, 0.85, 3.01, 4.19, 6.7 , 9.72, 9.91, 9.94],
       [1.

In [32]:
%%time
# compiled
foo0398 = exp(0.398, n=100)
foo0398.reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/1100 [00:00<?, ?it/s]

CPU times: user 1.83 s, sys: 721 ms, total: 2.55 s
Wall time: 1h 9min 53s


array([[ 2.41,  2.41,  2.42,  2.41,  2.39,  2.36,  2.29,  2.17,  2.  ,  1.75,  1.46,  1.13,  0.85,  0.67,  0.66],
       [ 2.42,  2.41,  2.42,  2.41,  2.39,  2.37,  2.31,  2.21,  2.07,  1.87,  1.63,  1.37,  1.14,  1.04,  1.02],
       [ 2.42,  2.42,  2.42,  2.41,  2.4 ,  2.39,  2.35,  2.28,  2.19,  2.06,  1.92,  1.76,  1.62,  1.57,  1.58],
       [ 2.42,  2.42,  2.42,  2.42,  2.42,  2.42,  2.41,  2.38,  2.38,  2.36,  2.37,  2.36,  2.36,  2.36,  2.39],
       [ 2.42,  2.42,  2.43,  2.43,  2.44,  2.47,  2.51,  2.55,  2.68,  2.84,  3.09,  3.3 ,  3.42,  3.46,  3.46],
       [ 2.42,  2.42,  2.44,  2.45,  2.48,  2.54,  2.64,  2.81,  3.14,  3.62,  4.2 ,  4.6 ,  4.75,  4.76,  4.8 ],
       [ 2.42,  2.43,  2.45,  2.47,  2.52,  2.62,  2.82,  3.15,  3.79,  4.71,  5.48,  5.86,  5.98,  6.02,  6.05],
       [ 2.41,  2.41,  2.4 ,  2.37,  2.31,  2.21,  2.01,  1.69,  1.11,  1.59,  2.3 ,  4.06,  7.49,  7.66,  7.71],
       [ 2.42,  2.41,  2.4 ,  2.37,  2.3 ,  2.19,  1.98,  1.59,  1.25,  1.96,  3.12,  7.

In [33]:
%%time
# compiled
foo0631 = exp(0.631, n=100)
foo0631.reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/1100 [00:00<?, ?it/s]

CPU times: user 1.71 s, sys: 618 ms, total: 2.33 s
Wall time: 1h 9min 55s


array([[ 3.32,  3.3 ,  3.28,  3.24,  3.18,  3.07,  2.8 ,  2.33,  1.66,  0.99,  0.74,  0.57,  0.6 ,  0.63,  0.65],
       [ 3.32,  3.3 ,  3.28,  3.25,  3.2 ,  3.11,  2.89,  2.52,  2.  ,  1.44,  1.19,  1.11,  1.09,  1.05,  1.05],
       [ 3.32,  3.3 ,  3.29,  3.26,  3.22,  3.15,  2.98,  2.72,  2.41,  2.  ,  1.84,  1.71,  1.65,  1.61,  1.6 ],
       [ 3.32,  3.31,  3.3 ,  3.28,  3.25,  3.22,  3.12,  2.98,  2.84,  2.65,  2.55,  2.46,  2.42,  2.4 ,  2.4 ],
       [ 3.32,  3.32,  3.32,  3.3 ,  3.31,  3.32,  3.32,  3.34,  3.4 ,  3.43,  3.49,  3.46,  3.49,  3.5 ,  3.5 ],
       [ 3.32,  3.33,  3.33,  3.34,  3.38,  3.46,  3.58,  3.81,  4.13,  4.5 ,  4.69,  4.75,  4.8 ,  4.82,  4.82],
       [ 3.32,  3.33,  3.35,  3.39,  3.46,  3.6 ,  3.86,  4.31,  4.95,  5.56,  5.82,  5.96,  6.02,  6.03,  6.06],
       [ 3.32,  3.32,  3.33,  3.33,  3.34,  3.36,  3.36,  4.05,  4.22,  4.16,  3.65,  6.37,  7.52,  7.64,  7.69],
       [ 3.32,  3.33,  3.34,  3.35,  3.38,  3.44,  3.7 ,  4.87,  5.09,  5.06,  4.65,  9.

In [34]:
%%time
# compiled
foo1000 = exp(1.000, n=100)
foo1000.reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/1100 [00:00<?, ?it/s]

CPU times: user 1.73 s, sys: 635 ms, total: 2.37 s
Wall time: 1h 9min 48s


array([[ 4.74,  4.73,  4.71,  4.7 ,  4.64,  4.54,  4.35,  3.98,  2.62,  1.06,  0.65,  0.49,  0.54,  0.5 ,  0.51],
       [ 4.74,  4.73,  4.72,  4.7 ,  4.65,  4.56,  4.38,  4.05,  3.15,  1.48,  1.09,  0.95,  0.89,  0.89,  0.96],
       [ 4.74,  4.74,  4.72,  4.71,  4.66,  4.58,  4.42,  4.16,  3.6 ,  2.33,  1.88,  1.63,  1.54,  1.58,  1.59],
       [ 4.74,  4.74,  4.72,  4.71,  4.68,  4.62,  4.5 ,  4.3 ,  3.99,  3.45,  2.96,  2.63,  2.47,  2.43,  2.42],
       [ 4.74,  4.74,  4.73,  4.73,  4.71,  4.68,  4.61,  4.5 ,  4.34,  4.1 ,  3.85,  3.62,  3.54,  3.52,  3.51],
       [ 4.74,  4.75,  4.74,  4.75,  4.75,  4.76,  4.76,  4.79,  4.83,  4.86,  4.87,  4.86,  4.85,  4.85,  4.84],
       [ 4.74,  4.75,  4.75,  4.78,  4.79,  4.85,  4.95,  5.11,  5.37,  5.68,  5.95,  6.07,  6.07,  6.07,  6.07],
       [ 4.74,  4.73,  4.71,  4.68,  4.6 ,  4.47,  4.19,  3.73,  3.01,  2.39,  1.7 ,  2.42,  7.45,  7.46,  7.58],
       [ 4.75,  4.73,  4.71,  4.69,  4.61,  4.47,  4.19,  3.73,  3.01,  3.5 ,  4.41,  4.

In [35]:
%%time
# compiled
foo1585 = exp(1.585, n=20)
foo1585.reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/220 [00:00<?, ?it/s]

CPU times: user 292 ms, sys: 321 ms, total: 613 ms
Wall time: 14min 10s


array([[5.96, 5.98, 5.95, 5.91, 5.83, 5.7 , 4.37, 3.07, 2.3 , 1.39, 0.88, 0.62, 0.54, 0.58, 0.61],
       [5.96, 5.98, 5.95, 5.91, 5.84, 5.73, 4.43, 3.58, 2.74, 1.77, 1.26, 0.95, 0.97, 1.  , 1.01],
       [5.96, 5.98, 5.95, 5.92, 5.85, 5.76, 4.97, 3.58, 3.18, 2.16, 1.79, 1.5 , 1.5 , 1.54, 1.59],
       [5.96, 5.98, 5.96, 5.93, 5.87, 5.8 , 5.44, 4.75, 3.66, 3.  , 2.32, 2.37, 2.4 , 2.41, 2.36],
       [5.96, 5.99, 5.97, 5.94, 5.9 , 5.84, 5.7 , 5.18, 4.6 , 4.1 , 3.67, 3.49, 3.5 , 3.49, 3.54],
       [5.96, 5.99, 5.98, 5.96, 5.94, 5.93, 5.87, 5.72, 5.5 , 5.21, 5.02, 4.84, 4.86, 4.84, 4.88],
       [5.96, 5.99, 5.99, 5.99, 5.98, 6.  , 6.  , 5.98, 6.03, 6.04, 6.07, 6.1 , 6.08, 6.09, 6.1 ],
       [5.96, 5.98, 5.96, 5.94, 5.88, 5.74, 5.4 , 4.84, 3.6 , 2.76, 2.38, 7.27, 7.45, 7.54, 7.48],
       [5.97, 5.98, 5.97, 5.95, 5.89, 5.75, 5.39, 4.86, 3.5 , 2.81, 3.97, 9.05, 9.35, 9.33, 9.33],
       [5.96, 5.98, 5.97, 5.95, 5.89, 5.75, 5.38, 4.86, 3.53, 2.83, 4.55, 9.47, 9.82, 9.92, 9.92],
       [5.

In [36]:
%%time
# compiled
foo2512 = exp(2.512, n=20)
foo2512.reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/220 [00:00<?, ?it/s]

CPU times: user 298 ms, sys: 311 ms, total: 610 ms
Wall time: 14min 7s


array([[ 8.12,  8.06,  7.99,  7.77,  7.44,  5.55,  1.12,  1.24,  0.9 ,  0.44,  0.42,  0.51,  0.56,  0.54,  0.58],
       [ 8.12,  8.06,  8.  ,  7.78,  7.46,  4.42,  0.97,  1.51,  1.06,  0.92,  0.76,  0.77,  0.76,  0.94,  0.96],
       [ 8.12,  8.06,  8.  ,  7.78,  7.47,  4.63,  2.04,  1.44,  1.31,  1.17,  1.1 ,  1.18,  1.2 ,  1.4 ,  1.48],
       [ 8.12,  8.07,  8.  ,  7.79,  7.48,  5.32,  2.88,  1.99,  1.72,  1.64,  1.85,  1.91,  2.1 ,  2.32,  2.35],
       [ 8.12,  8.07,  8.01,  7.81,  7.51,  6.  ,  3.88,  2.3 ,  2.03,  2.27,  2.56,  2.8 ,  3.06,  3.4 ,  3.49],
       [ 8.12,  8.09,  8.04,  7.84,  7.55,  6.97,  5.47,  2.65,  2.69,  2.76,  3.35,  4.12,  4.67,  4.76,  4.84],
       [ 8.12,  8.08,  8.05,  7.86,  7.57,  7.17,  6.34,  4.15,  3.4 ,  3.81,  4.41,  5.55,  5.88,  6.02,  6.09],
       [ 8.12,  8.14,  8.16,  8.12,  8.12,  8.09,  8.11,  7.89,  7.78,  7.71,  7.68,  7.72,  7.66,  7.68,  7.7 ],
       [ 8.12,  8.16,  8.2 ,  8.17,  8.21,  8.29,  8.48,  8.57,  8.87,  9.22,  9.39,  9.

In [37]:
%%time
# compiled
foo3981 = exp(3.981, n=20)
foo3981.reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/220 [00:00<?, ?it/s]

CPU times: user 288 ms, sys: 278 ms, total: 566 ms
Wall time: 14min 8s


array([[ 9.43,  9.26,  9.12,  8.4 ,  7.39,  4.89,  2.78,  0.33,  0.26,  0.14,  0.2 ,  0.29,  0.34,  0.41,  0.49],
       [ 9.43,  9.26,  9.13,  8.51,  7.4 ,  5.58,  3.35,  0.75,  0.54,  0.39,  0.47,  0.62,  0.69,  0.77,  0.96],
       [ 9.43,  9.26,  9.12,  8.41,  7.42,  6.05,  3.95,  1.12,  0.77,  0.61,  0.77,  0.89,  1.05,  1.25,  1.51],
       [ 9.43,  9.27,  9.13,  8.7 ,  7.22,  6.22,  4.13,  1.39,  1.01,  1.01,  1.31,  1.59,  1.97,  2.17,  2.32],
       [ 9.43,  9.28,  9.14,  8.34,  7.29,  6.32,  4.77,  2.41,  1.61,  1.59,  2.  ,  2.42,  2.97,  3.41,  3.41],
       [ 9.43,  9.3 ,  9.17,  8.56,  7.24,  6.43,  5.  ,  3.21,  2.59,  2.48,  2.88,  3.58,  4.5 ,  4.73,  4.77],
       [ 9.43,  9.3 ,  9.19,  8.61,  7.07,  6.34,  5.45,  3.46,  3.19,  3.14,  3.74,  5.27,  5.86,  5.99,  6.06],
       [ 9.43,  9.39,  9.39,  9.35,  9.32,  9.12,  8.84,  8.24,  7.77,  7.63,  7.58,  7.72,  7.68,  7.71,  7.66],
       [ 9.43,  9.42,  9.43,  9.42,  9.45,  9.39,  9.42,  9.34,  9.32,  9.38,  9.37,  9.

In [38]:
%%time
# compiled
foo6310 = exp(6.310, n=20)
foo6310.reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/220 [00:00<?, ?it/s]

CPU times: user 291 ms, sys: 305 ms, total: 596 ms
Wall time: 14min 7s


array([[ 9.94,  8.52,  6.47,  3.54,  1.93,  1.01,  0.38, -0.13,  0.36,  0.43,  0.41,  0.55,  0.56,  0.62,  0.59],
       [ 9.94,  8.55,  6.36,  3.51,  2.11,  0.96,  0.52, -0.14,  0.66,  0.64,  0.6 ,  0.69,  0.79,  0.83,  0.88],
       [ 9.94,  8.49,  6.28,  3.38,  2.22,  1.11,  0.33,  0.13,  0.6 ,  1.1 ,  1.11,  1.21,  1.43,  1.32,  1.54],
       [ 9.94,  8.47,  6.16,  3.29,  2.33,  1.37,  0.74,  0.77,  1.2 ,  1.53,  1.8 ,  1.94,  2.17,  2.1 ,  2.25],
       [ 9.94,  8.63,  5.88,  3.36,  2.56,  1.65,  1.57,  1.37,  1.84,  2.26,  2.63,  3.03,  3.32,  3.39,  3.5 ],
       [ 9.94,  8.11,  6.43,  3.51,  2.99,  2.71,  2.71,  2.35,  2.83,  3.28,  3.98,  4.51,  4.84,  4.77,  4.84],
       [ 9.95,  8.87,  4.52,  3.13,  3.1 ,  3.3 ,  3.13,  2.81,  3.53,  4.1 ,  5.43,  6.  ,  5.99,  6.04,  5.98],
       [ 9.94,  9.9 ,  9.88,  9.73,  9.3 ,  8.31,  8.04,  7.55,  7.57,  7.52,  7.41,  7.53,  7.73,  7.76,  7.76],
       [ 9.94,  9.93,  9.94,  9.92,  9.9 ,  9.76,  9.6 ,  9.29,  9.32,  9.4 ,  9.41,  9.

In [39]:
%%time
# compiled
foo10000 = exp(10.000, n=20)
foo10000.reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/220 [00:00<?, ?it/s]

CPU times: user 348 ms, sys: 314 ms, total: 662 ms
Wall time: 14min 7s


array([[ 9.98,  3.79,  2.65,  3.46,  0.27, -0.64,  0.72, -0.03,  0.73,  0.53,  0.45,  0.45,  0.5 ,  0.44,  0.5 ],
       [ 9.98,  3.76,  2.77,  3.49,  1.69, -0.18,  0.87,  0.38,  0.67,  0.63,  0.95,  0.76,  0.75,  0.86,  0.88],
       [ 9.98,  3.89,  2.79,  4.07,  1.51,  0.42,  0.5 ,  1.11,  1.06,  1.19,  1.06,  1.16,  1.3 ,  1.4 ,  1.48],
       [ 9.98,  3.54,  4.08,  4.37,  2.44,  1.49,  1.08,  0.74,  1.56,  1.65,  1.86,  2.16,  2.05,  2.25,  2.3 ],
       [ 9.98,  6.11,  3.54,  4.06,  3.08,  2.  ,  1.9 ,  1.37,  1.7 ,  2.3 ,  2.91,  3.12,  3.13,  3.3 ,  3.42],
       [ 9.98,  5.3 ,  3.7 ,  2.82,  3.98,  3.18,  2.98,  2.61,  2.98,  3.74,  4.24,  4.46,  4.65,  4.64,  4.78],
       [ 9.98,  2.73,  2.77,  3.26,  3.93,  3.79,  3.6 ,  3.49,  4.05,  4.81,  5.52,  5.79,  5.96,  5.95,  6.01],
       [ 9.98,  3.69,  8.76,  8.52,  8.2 ,  7.41,  7.13,  6.92,  6.85,  6.94,  7.17,  7.22,  7.09,  7.44,  7.51],
       [ 9.98,  9.74, 10.  ,  9.98,  9.2 ,  9.86,  9.49,  8.57,  9.01,  9.26,  9.16,  9.